In [2]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
client_id = 'sh-29927974-7d1f-4f2e-9751-f59a8149a944'
client_secret = 'Zqrx0QU4ivjOl3TM77zsm0Hhib8QWivO'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

b'[]'


In [24]:
from sentinelhub import SentinelHubRequest, SHConfig

cfg = SHConfig()
cfg.sh_client_id = 'sh-29927974-7d1f-4f2e-9751-f59a8149a944'
cfg.sh_client_secret = 'Zqrx0QU4ivjOl3TM77zsm0Hhib8QWivO'

print("Credentials OK" if cfg.sh_client_id and cfg.sh_client_secret else "Missing credentials")


Credentials OK


In [25]:
# Step 1: Create and save configuration properly
from sentinelhub import SHConfig
import getpass

# Create configuration for Copernicus Data Space Ecosystem
config = SHConfig()

cfg.sh_client_id = 'sh-29927974-7d1f-4f2e-9751-f59a8149a944'
cfg.sh_client_secret = 'Zqrx0QU4ivjOl3TM77zsm0Hhib8QWivO'

# IMPORTANT: Set the correct URLs for Copernicus Data Space Ecosystem
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

# Save this configuration to a profile
config.save("cdse")

print("Configuration saved to 'cdse' profile")
print("Current config:")
print(config)

# Step 2: Verify the configuration is saved correctly
config_check = SHConfig("cdse")
print("\nLoaded config from 'cdse' profile:")
print(config_check)
print(f"Client ID set: {bool(config_check.sh_client_id)}")
print(f"Client Secret set: {bool(config_check.sh_client_secret)}")
print(f"Base URL: {config_check.sh_base_url}")
print(f"Token URL: {config_check.sh_token_url}")

Configuration saved to 'cdse' profile
Current config:
{
  "instance_id": "",
  "sh_client_id": "",
  "sh_client_secret": "",
  "sh_base_url": "https://sh.dataspace.copernicus.eu",
  "sh_auth_base_url": null,
  "sh_token_url": "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
  "geopedia_wms_url": "https://service.geopedia.world",
  "geopedia_rest_url": "https://www.geopedia.world/rest",
  "aws_access_key_id": "",
  "aws_secret_access_key": "",
  "aws_session_token": "",
  "aws_metadata_url": "https://roda.sentinel-hub.com",
  "aws_s3_l1c_bucket": "sentinel-s2-l1c",
  "aws_s3_l2a_bucket": "sentinel-s2-l2a",
  "opensearch_url": "http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2",
  "max_wfs_records_per_query": 100,
  "max_opensearch_records_per_query": 500,
  "max_download_attempts": 4,
  "download_sleep_time": 5.0,
  "download_timeout_seconds": 120.0,
  "number_of_download_processes": 1,
  "max_retries": null
}

Loaded config fro

In [23]:
from sentinelhub import (
    SHConfig, CRS, BBox, DataCollection, SentinelHubRequest, 
    MimeType, bbox_to_dimensions
)
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Load the saved configuration
try:
    # Try to load the saved 'cdse' profile
    config = SHConfig("cdse")
    print(f"Loaded config from 'cdse' profile")
    print(f"Client ID set: {bool(config.sh_client_id)}")
    print(f"Client Secret set: {bool(config.sh_client_secret)}")
    print(f"Base URL: {config.sh_base_url}")
    print(f"Token URL: {config.sh_token_url}")
    
    # Check if credentials are actually set
    if not config.sh_client_id or not config.sh_client_secret:
        raise ValueError("Credentials not found in saved profile")
        
except Exception as e:
    print(f"Error loading saved config: {e}")
    print("Creating new configuration...")
    
    # Create new configuration if profile doesn't exist
    config = SHConfig()
    config.sh_client_id = 'YOUR_ACTUAL_CLIENT_ID'  # REPLACE THIS!
    config.sh_client_secret = 'YOUR_ACTUAL_CLIENT_SECRET'  # REPLACE THIS!
    config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    config.sh_base_url = "https://sh.dataspace.copernicus.eu"
    
    # Save for future use
    config.save("cdse")
    print("New configuration saved")

# Step 2: Verify configuration before proceeding
if not config.sh_client_id or not config.sh_client_secret:
    print("ERROR: You must set your actual OAuth credentials!")
    print("Replace 'YOUR_ACTUAL_CLIENT_ID' and 'YOUR_ACTUAL_CLIENT_SECRET' with your real values")
    exit()

print(f"\nUsing configuration:")
print(f"Base URL: {config.sh_base_url}")
print(f"Token URL: {config.sh_token_url}")
print(f"Client ID: {config.sh_client_id[:8]}..." if config.sh_client_id else "Not set")

# Step 3: Define your area (2km x 2km around your point)
# Your coordinates: 50°17'44.5"N 36°39'42.9"E = 50.2957°N, 36.6619°E
bbox_coords = [36.6489, 50.2867, 36.6749, 50.3047]  # [lon_min, lat_min, lon_max, lat_max]
bbox = BBox(bbox=bbox_coords, crs=CRS.WGS84)
size = bbox_to_dimensions(bbox, resolution=10)  # 10m resolution for Sentinel-2

print(f"\nImage size at 10m resolution: {size} pixels")

# Step 4: Define evaluation script for true color
evalscript = """
//VERSION=3
function setup() {
    return {
        input: ["B02", "B03", "B04"],
        output: { bands: 3 }
    };
}
function evaluatePixel(sample) {
    return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
}
"""

# Step 5: Create request with explicit config
request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=('2024-01-01', '2024-12-31'),
            mosaicking_order='leastCC'
        )
    ],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    bbox=bbox,
    size=size,
    config=config  # Explicitly pass the config!
)

# Step 6: Download with error handling
try:
    print("\nStarting download...")
    image = request.get_data()[0]
    print(f"Successfully downloaded image with shape: {image.shape}")
    
    # Step 7: Display and save the image
    plt.figure(figsize=(12, 10))
    plt.imshow(image)
    plt.title("Sentinel-2 Image - 50°17'44.5\"N 36°39'42.9\"E (2km x 2km)")
    plt.axis('off')
    plt.tight_layout()
    
    # Save the image
    plt.savefig('sentinel2_ukraine_location.png', dpi=300, bbox_inches='tight')
    print("Image saved as 'sentinel2_ukraine_location.png'")
    plt.show()
    
    # Additional info
    print(f"\nImage details:")
    print(f"Shape: {image.shape}")
    print(f"Data type: {image.dtype}")
    print(f"Value range: {image.min()} - {image.max()}")
    
except Exception as e:
    print(f"Error during download: {e}")
    print("\nTroubleshooting tips:")
    print("1. Make sure you replaced 'YOUR_ACTUAL_CLIENT_ID' and 'YOUR_ACTUAL_CLIENT_SECRET'")
    print("2. Verify your OAuth client is created for Copernicus Data Space Ecosystem")
    print("3. Check that your OAuth client hasn't expired")
    print("4. Try a different time range if no data is available")

Loaded config from 'cdse' profile
Client ID set: True
Client Secret set: True
Base URL: https://sh.dataspace.copernicus.eu
Token URL: https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token

Using configuration:
Base URL: https://sh.dataspace.copernicus.eu
Token URL: https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token
Client ID: YOUR_ACT...

Image size at 10m resolution: (191, 194) pixels

Starting download...
Error during download: (invalid_client) Invalid client or Invalid client credentials

Troubleshooting tips:
1. Make sure you replaced 'YOUR_ACTUAL_CLIENT_ID' and 'YOUR_ACTUAL_CLIENT_SECRET'
2. Verify your OAuth client is created for Copernicus Data Space Ecosystem
3. Check that your OAuth client hasn't expired
4. Try a different time range if no data is available


In [14]:
from sentinelhub import (
    SHConfig, CRS, BBox, DataCollection, SentinelHubRequest, 
    MimeType, bbox_to_dimensions
)

# Step 1: Configure for Copernicus Data Space Ecosystem
config = SHConfig()
cfg.sh_client_id = 'sh-29927974-7d1f-4f2e-9751-f59a8149a944'
cfg.sh_client_secret = 'Zqrx0QU4ivjOl3TM77zsm0Hhib8QWivO'
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

# Optionally save this configuration for future use
config.save('cdse')

# Step 2: Define your area (2km x 2km around your point)
# Your coordinates: 50°17'44.5"N 36°39'42.9"E = 50.2957°N, 36.6619°E
bbox_coords = [36.6489, 50.2867, 36.6749, 50.3047]  # [lon_min, lat_min, lon_max, lat_max]
bbox = BBox(bbox=bbox_coords, crs=CRS.WGS84)
size = bbox_to_dimensions(bbox, resolution=10)  # 10m resolution for Sentinel-2

print(f"Image size at 10m resolution: {size} pixels")

# Step 3: Define evaluation script for true color
evalscript = """
//VERSION=3
function setup() {
    return {
        input: ["B02", "B03", "B04"],
        output: { bands: 3 }
    };
}
function evaluatePixel(sample) {
    return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
}
"""

# Step 4: Create request
request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=('2024-01-01', '2024-12-31'),
            mosaicking_order='leastCC'
        )
    ],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    bbox=bbox,
    size=size,
    config=config  # Make sure to pass the config!
)

# Step 5: Download
try:
    image = request.get_data()[0]
    print(f"Successfully downloaded image with shape: {image.shape}")
    
    # Save the image
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title("Sentinel-2 Image - 50°17'44.5\"N 36°39'42.9\"E")
    plt.axis('off')
    plt.savefig('sentinel2_image.png', dpi=300, bbox_inches='tight')
    plt.show()
    
except Exception as e:
    print(f"Error: {e}")

Image size at 10m resolution: (191, 194) pixels
Error: Configuration parameters 'sh_client_id' and 'sh_client_secret' have to be set in order to authenticate with Sentinel Hub service. Check https://sentinelhub-py.readthedocs.io/en/latest/configure.html for more info.


In [4]:
from sentinelhub import SHConfig, SentinelHubCatalog

cfg = SHConfig()

# Your client credentials
cfg.sh_client_id  = 'sh-29927974-7d1f-4f2e-9751-f59a8149a944'
cfg.sh_client_secret = 'Zqrx0QU4ivjOl3TM77zsm0Hhib8QWivO'

cfg.sh_auth_base_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
cfg.sh_base_url = "https://creodias.sentinel-hub.com"

# Just try an authenticated request
catalog = SentinelHubCatalog(config=cfg)
results = list(catalog.search(collection='sentinel-2-l2a', bbox=[36.6, 50.2, 36.7, 50.3], time=('2025-05-01', '2025-05-24')))

print(f"Found {len(results)} results.")


AttributeError: 'list' object has no attribute 'crs'

In [6]:
from sentinelhub import SHConfig, SentinelHubCatalog, BBox, CRS

cfg = SHConfig()
# Your client credentials
cfg.sh_client_id  = 'sh-29927974-7d1f-4f2e-9751-f59a8149a944'
cfg.sh_client_secret = 'Zqrx0QU4ivjOl3TM77zsm0Hhib8QWivO'
cfg.sh_auth_base_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
cfg.sh_base_url      = "https://creodias.sentinel-hub.com"

# Define the BBox properly
bbox = BBox(bbox=[36.6, 50.2, 36.7, 50.3], crs=CRS.WGS84)

catalog = SentinelHubCatalog(config=cfg)

# Now this works
results = list(catalog.search(
    collection='sentinel-2-l2a',
    bbox=bbox,
    time=('2025-05-01', '2025-05-24')
))

print(f"✅ Found {len(results)} results.")


InvalidClientError: (invalid_client) Invalid client or Invalid client credentials

In [26]:
# Debug and Setup Script for Sentinel Hub Authentication
from sentinelhub import SHConfig
import os
import requests

print("=== SENTINEL HUB AUTHENTICATION SETUP ===\n")

# Step 1: Check if you have the correct credentials
print("Step 1: Let's set up your credentials properly")
print("You need to get these from: https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings")
print()

# Get credentials from user input (replace with your actual values)
client_id = input("Enter your OAuth Client ID: ").strip()
client_secret = input("Enter your OAuth Client Secret: ").strip()

if not client_id or not client_secret:
    print("❌ ERROR: You must provide both Client ID and Client Secret!")
    print("Go to https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings to create OAuth client")
    exit()

print(f"\n✅ Credentials entered:")
print(f"Client ID: {client_id[:8]}...{client_id[-4:] if len(client_id) > 12 else client_id}")
print(f"Client Secret: {'*' * len(client_secret)}")

# Step 2: Test authentication directly with API
print("\nStep 2: Testing authentication with API...")

token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

try:
    response = requests.post(token_url, data=data, timeout=30)
    
    if response.status_code == 200:
        token_data = response.json()
        print("✅ Authentication successful!")
        print(f"Access token received (expires in {token_data.get('expires_in', 'unknown')} seconds)")
        auth_working = True
    else:
        print(f"❌ Authentication failed!")
        print(f"Status code: {response.status_code}")
        print(f"Response: {response.text}")
        auth_working = False
        
except Exception as e:
    print(f"❌ Network error during authentication: {e}")
    auth_working = False

if not auth_working:
    print("\n🔧 Troubleshooting:")
    print("1. Check your Client ID and Secret are correct")
    print("2. Verify your OAuth client hasn't expired")
    print("3. Ensure you created the OAuth client in Copernicus Data Space Ecosystem (not regular Sentinel Hub)")
    exit()

# Step 3: Configure SentinelHub
print("\nStep 3: Configuring SentinelHub Python package...")

config = SHConfig()
config.sh_client_id = client_id
config.sh_client_secret = client_secret
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

# Save configuration
config.save("cdse")
print("✅ Configuration saved to 'cdse' profile")

# Step 4: Verify saved configuration
print("\nStep 4: Verifying saved configuration...")
test_config = SHConfig("cdse")

print(f"✅ Configuration verification:")
print(f"Client ID matches: {test_config.sh_client_id == client_id}")
print(f"Client Secret matches: {test_config.sh_client_secret == client_secret}")
print(f"Base URL: {test_config.sh_base_url}")
print(f"Token URL: {test_config.sh_token_url}")

# Step 5: Test with SentinelHub
print("\nStep 5: Testing with SentinelHub package...")

try:
    from sentinelhub import SentinelHubSession
    session = SentinelHubSession(config=test_config)
    print("✅ SentinelHub session created successfully!")
    print("🎉 You're ready to download satellite data!")
    
    # Show final configuration
    print(f"\n=== FINAL CONFIGURATION ===")
    print(f"Profile name: 'cdse'")
    print(f"Client ID: {client_id[:8]}...")
    print(f"Base URL: {test_config.sh_base_url}")
    print(f"Configuration file location: {SHConfig.get_config_location()}")
    
except Exception as e:
    print(f"❌ SentinelHub session test failed: {e}")
    print("This might be a temporary issue. Your credentials are working with the API.")

print("\n=== NEXT STEPS ===")
print("Now you can run your satellite data download script!")
print("The configuration is saved and will work automatically.")

=== SENTINEL HUB AUTHENTICATION SETUP ===

Step 1: Let's set up your credentials properly
You need to get these from: https://shapps.dataspace.copernicus.eu/dashboard/#/account/settings



: 